[View in Colaboratory](https://colab.research.google.com/github/restrepo/WOSplus/blob/master/test_sample.ipynb)

In [0]:
%%bash
# Check if colaboratory was launched and install missing requirements
if [ "$(pwd)" == /content ];then
    pip install openpyxl xlrd unidecode python-levenshtein requests_testadapter > /dev/null
    git clone https://github.com/restrepo/WOSplus.git > /dev/null
    mv WOSplus/wosplus/* .
fi

In [0]:
import wosplus

In [12]:
%%writefile drive.cfg
[FILES]
Sample_WOS.xlsx = 1--LJZ4mYyQcaJ93xBdbnYj-ZzdjO2Wq2
Sample_SCI.xlsx = 1-3a-hguQTk5ko8JRLCx--EKaslxGVscf
Sample_SCP.xlsx = 1-IAWlMdp2U-9L2jvZUio04ub1Ym3PX-H

Overwriting drive.cfg


In [13]:
cib=wosplus.wosplus('drive.cfg')

cib.load_biblio('Sample_WOS.xlsx')
cib.load_biblio('Sample_SCI.xlsx',prefix='SCI')
cib.load_biblio('Sample_SCP.xlsx',prefix='SCP')

print('before merge: {}'.format( cib.WOS.shape[0]+cib.SCI.shape[0]+cib.SCP.shape[0] )  )

cib.merge(left='WOS',right='SCI')

if True:
    print('intial: {}'.format( cib.WOS.shape[0]+cib.SCI.shape[0]) )
    print('final : {}'.format(  cib.WOS_SCI.shape) )

cib.merge(left='WOS_SCI',right='SCP')

if True:
    print('intial: {}'.format( cib.WOS_SCI.shape[0]+cib.SCP.shape[0]) )
    print('final : {}'.format(  cib.WOS_SCI_SCP.shape) )

/content/_wos_scp.py:54: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  return df.rename_axis( dict( (key,prefix+'_'+key) for key in df.columns.values) , axis=1)


before merge: 48
.intial: 38
final : (28, 96)
..intial: 38
final : (30, 142)
